# 0 - Librerías y variables

In [1]:
# Librerías
# ------------------------------------------------------------------------------
import os
import requests
import json

from dotenv import load_dotenv, dotenv_values
load_dotenv()

True

In [2]:
# Variables
# ------------------------------------------------------------------------------
env_vars = dotenv_values()
for key in env_vars.keys():
    print(key)

OPENAI_API_KEY
PROXYCURL_API_KEY
TAVILY_API_KEY
LANGCHAIN_TRACING_V2
LANGCHAIN_ENDPOINT
LANGCHAIN_API_KEY
LANGCHAIN_PROJECT


# 1 - Introducción a los RAG

RAG significa Retrieval-Augmented Generation, o Generación Aumentada por Recuperación. Es una arquitectura híbrida que combina LLMs con motores de recuperación de información, generalmente basados en embeddings y búsqueda vectorial.

La idea clave es no depender solo del conocimiento interno del LLM, sino complementarlo con información externa relevante que se recupera en tiempo real desde una base documental.

**¿Por qué usar RAG?**

Los modelos generativos, aunque potentes, tienen tres limitaciones clave:
- Alucinaciones: inventan respuestas cuando no saben algo.
- Obsolescencia: su conocimiento está limitado al momento en que fueron entrenados.
- Costo de fine-tuning: actualizar su conocimiento requiere reentrenar o usar técnicas más complejas.

RAG soluciona esto al buscar primero información en una base de datos externa (documentos, PDFs, páginas web, etc.), y luego alimentar esa información como contexto al modelo generativo, que produce una respuesta más precisa, basada en datos.

**¿Cómo funciona RAG? (flujo típico)**

Un sistema RAG moderno, como el que puedes construir con LangChain, suele organizarse en las siguientes etapas:

- Load (Carga de documentos): Se ingestan las fuentes de información que van a alimentar al sistema. El objetivo es leer los datos en bruto y convertirlos en texto plano.
- Split (Segmentación de texto): Los textos se dividen en fragmentos más pequeños para facilitar su uso como contexto. Se usan técnicas de ventana deslizante con solapamiento para preservar la coherencia semántica.
- Storage (Vectorización y almacenamiento): Cada fragmento se convierte en un embedding y se almacena en una base vectorial para búsquedas por similitud semántica.
- Retrieval (Recuperación): La consulta del usuario se vectoriza y se buscan los fragmentos más similares en la base vectorial. Estos fragmentos forman el contexto que se pasará al modelo generativo.
- Output (Generación de respuesta):Se combinan el contexto recuperado y la consulta original, y se pasan al modelo de lenguaje, que genera una respuesta informada.


![](../img/RAG.png)

# 2 - Document Loading

In [22]:
from langchain.document_loaders import PyPDFLoader, WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


## 2.1. - PDF

In [7]:
# Instance PDF loader
loader = PyPDFLoader("../data/docs/cs229_lectures/MachineLearning-Lecture01.pdf")

# Load PDF
pages = loader.load()

Cada página es un `Document`. Cada `Document` contiene contenido `page_content` y metadata `metadata`.

In [9]:
len(pages)

22

In [21]:
page = pages[0]

print(type(page))
page

<class 'langchain_core.documents.base.Document'>


Document(metadata={'producer': 'Acrobat Distiller 8.1.0 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'creationdate': '2008-07-11T11:25:23-07:00', 'author': '', 'moddate': '2008-07-11T11:25:23-07:00', 'title': '', 'source': '../data/docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'total_pages': 22, 'page': 0, 'page_label': '1'}, page_content='MachineLearning-Lecture01  \nInstructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is just spend a little time going over the logistics \nof the class, and then we\'ll start to talk a bit about machine learning.  \nBy way of introduction, my name\'s Andrew Ng and I\'ll be instructor for this class. And so \nI personally work in machine learning, and I\'ve worked on it for about 15 years now, and \nI actually think that machine learning is the most exciting field of all the computer \nsciences. So I\'m actually always excited about teaching this class. Sometimes I actually \n

In [20]:
page.metadata

{'producer': 'Acrobat Distiller 8.1.0 (Windows)',
 'creator': 'PScript5.dll Version 5.2.2',
 'creationdate': '2008-07-11T11:25:23-07:00',
 'author': '',
 'moddate': '2008-07-11T11:25:23-07:00',
 'title': '',
 'source': '../data/docs/cs229_lectures/MachineLearning-Lecture01.pdf',
 'total_pages': 22,
 'page': 0,
 'page_label': '1'}

In [18]:
page.page_content

'MachineLearning-Lecture01  \nInstructor (Andrew Ng): Okay. Good morning. Welcome to CS229, the machine \nlearning class. So what I wanna do today is just spend a little time going over the logistics \nof the class, and then we\'ll start to talk a bit about machine learning.  \nBy way of introduction, my name\'s Andrew Ng and I\'ll be instructor for this class. And so \nI personally work in machine learning, and I\'ve worked on it for about 15 years now, and \nI actually think that machine learning is the most exciting field of all the computer \nsciences. So I\'m actually always excited about teaching this class. Sometimes I actually \nthink that machine learning is not only the most exciting thing in computer science, but \nthe most exciting thing in all of human endeavor, so maybe a little bias there.  \nI also want to introduce the TAs, who are all graduate students doing research in or \nrelated to the machine learning and all aspects of machine learning. Paul Baumstarck \nworks i

## 2.2. URLs

In [24]:
# Instanciar loader
loader = WebBaseLoader("https://github.com/basecamp/handbook/blob/master/titles-for-programmers.md")

# Load URL
docs = loader.load()

In [25]:
len(docs)

1

In [26]:
doc = docs[0]

print(type(doc))
doc

<class 'langchain_core.documents.base.Document'>


Document(metadata={'source': 'https://github.com/basecamp/handbook/blob/master/titles-for-programmers.md', 'title': 'handbook/titles-for-programmers.md at master · basecamp/handbook · GitHub', 'description': 'Basecamp Employee Handbook. Contribute to basecamp/handbook development by creating an account on GitHub.', 'language': 'en'}, page_content='\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nhandbook/titles-for-programmers.md at master · basecamp/handbook · GitHub\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nNavigation Menu\n\nToggle navigation\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n            Sign in\n          \n\n\n \n\n\nAppearance settings\n\n\n\n\n\n\n\n\n\n\n\n        Product\n        \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n            GitHub Copilot\n          \n        Write better 